In [1]:
import numpy as np
import xarray as xr
from seapopym_optimization.cost_function import cost_function

In [2]:
from seapopym.model.no_transport_model import NoTransportModel

from seapopym.configuration.no_transport.configuration import NoTransportConfiguration

from seapopym.configuration.no_transport.parameter import (
    ForcingParameters,
    FunctionalGroups,
    KernelParameters,
    NoTransportParameters,
)

from seapopym.configuration.parameters.parameter_forcing import ForcingUnit

from seapopym.configuration.parameters.parameter_functional_group import (
    FunctionalGroupUnit,
    FunctionalGroupUnitMigratoryParameters,
    FunctionalGroupUnitRelationParameters,
)

from seapopym.standard.units import StandardUnitsLabels

Load forcing.


In [3]:
time_start, time_end = "1998-01-01", "2022-01-01"

hot_data = xr.open_dataset("../notebooks/1_data_processing/1_1_Forcing/bats_cmems.zarr", engine="zarr")
hot_data["T"].attrs["units"] = StandardUnitsLabels.temperature.units
hot_data = hot_data.sel(time=slice(time_start, time_end))
_ = hot_data.load()

cafe_npp = xr.open_dataset("../notebooks/1_data_processing/1_1_Forcing/bats_cafe.zarr", engine="zarr")
cafe_npp = cafe_npp.sel(time=slice(time_start, time_end))
cafe_npp = cafe_npp.dropna("time")
cafe_npp = cafe_npp.resample(time="D").interpolate("linear")
cafe_npp.lat.attrs = {"units": "degrees_north", "long_name": "latitude", "axis": "Y"}
cafe_npp.lon.attrs = {"units": "degrees_east", "long_name": "longitude", "axis": "X"}
cafe_npp = cafe_npp.rename({"lat": "latitude", "lon": "longitude"})
_ = cafe_npp.load()

hot_obs = xr.open_dataset("../notebooks/1_data_processing/1_1_Forcing/bats_obs.zarr", engine="zarr")
hot_obs = hot_obs.sel(time=slice(time_start, time_end))
_ = hot_obs.load()

Create structure for SeapoPym simulation.


In [4]:
forcing_parameters = ForcingParameters(
    temperature=ForcingUnit(forcing=hot_data["T"], resolution=1/12),
    primary_production=ForcingUnit(forcing=cafe_npp["CAFE"], resolution=1/12),
)

2024-09-24 09:11:11,526 :: Seapodym ::  WARNING ::
|	CAFE unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



In [5]:
cost_function(
    args=np.array([[0.1,0.1,0.1,0.4,0.5,0.6, 0.7]]),
    groups_name=["D1N1"],
    nb_parameters=7,
    forcing_parameters=forcing_parameters,
    observations=...,
)

2024-09-24 09:11:11,533 :: Seapodym ::  WARNING ::
|	Parameter temperature_recruitment_rate : 0.2 has a positive value. It means that the recruitment time is increasing with temperature. Do you mean to use a negative value?

2024-09-24 09:11:11,545 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-09-24 09:11:11,562 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.



KeyError: "not all values found in index 'depth'. Try setting the `method` keyword argument (example: method='nearest')."